In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import rtde_control
import time

rtde_c = rtde_control.RTDEControlInterface("169.254.37.182")
rtde_c.moveL([-0.00772, -0.64201, 0.19098, 0.032, -3.154, -0.017], 0.5, 0.3)
def distance(event,x,y,flags,param):
    if event==cv2.EVENT_LBUTTONDOWN:
        print(event)
        cv2.circle(capture,(x,y),2,(128,0,128),-1)
        print((x,y))
        d=depth_frame.get_distance(int(x),int(y))
        print(d)
        if (d==0.0) :
            print('No depth info, You must use an window')
        else:
            x_w, y_w, z_w = convert_depth_to_phys_coord_using_realsense(int(x),int(y), d, camera_info)
            print('Camera Points are:',x_w, y_w, z_w)
            p1=f'{x_w} {y_w} {z_w}'
            p = [float(value) for value in p1.split(' ')]
            p.append(1.0)
            dst_p=np.matmul(k,p)
            dst_p=dst_p.tolist()
            print(dst_p)
            rtde_c.moveL([dst_p[0]/1000.0, dst_p[1]/1000.0, dst_p[2]/1000.0, 0.032, -3.154, -0.017], 0.5, 0.3)
            #time.sleep(5)
            #dst_p=(dst_p[0])[:-1]
            #print(dst_p)
            del p1
            del p

def convert_depth_to_phys_coord_using_realsense(x, y, depth, cameraInfo):
    _intrinsics = rs.intrinsics()
    _intrinsics.width = cameraInfo.width
    _intrinsics.height = cameraInfo.height
    _intrinsics.ppx = cameraInfo.ppx
    _intrinsics.ppy = cameraInfo.ppy
    _intrinsics.fx = cameraInfo.fx
    _intrinsics.fy = cameraInfo.fy
    # _intrinsics.model = cameraInfo.distortion_model
    _intrinsics.model  = rs.distortion.none
    _intrinsics.coeffs = [i for i in cameraInfo.coeffs]
    result = rs.rs2_deproject_pixel_to_point(_intrinsics, [x, y], depth)
    # result[0]: right, result[1]: down, result[2]: forward
    return result[0], -result[1], -result[2]
"""
# Setup the pipeline
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipe.start(cfg)
"""
k= [[-9.79365051e+02,6.22173726e-01,-1.81205747e+02,-1.09894855e+01],[2.63002626e+00,-9.88050938e+02,-2.41157019e+01,-6.93151121e+02],[1.52250424e+02,7.62100894e+01,-4.30734272e+02,4.48864321e+01],[0.00000000e+00,0.00000000e+00,0.00000000e+00,1.00000000e+00]]

# Create a pipeline
pipeline = rs.pipeline()

# Create a config and configure the pipeline to stream
config = rs.config()
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))
print(device_product_line)
found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        print("There is a depth camera with color sensor")
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 848, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)


# Setup the 'High Accuracy'-mode
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)
clipping_distance_in_meters = 0.3 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale
print(clipping_distance)
preset_range = depth_sensor.get_option_range(rs.option.visual_preset)
for i in range(int(preset_range.max)):
    visulpreset = depth_sensor.get_option_value_description(rs.option.visual_preset,i)
    print('%02d: %s'%(i,visulpreset))
    if visulpreset == "High Accuracy":
        depth_sensor.set_option(rs.option.visual_preset, i)
# enable higher laser-power for better detection
depth_sensor.set_option(rs.option.laser_power, 180)
# lower the depth unit for better accuracy and shorter distance covered
depth_sensor.set_option(rs.option.depth_units, 0.0005)
align_to = rs.stream.color
align = rs.align(align_to)
# Skip first frames for auto-exposure to adjust
for x in range(5):
    pipeline.wait_for_frames()
try:
    while True:

        # Stores next frameset
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()

        if color_frame:
            aligned_frames = align.process(frames)

            # Get aligned frames
            aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()

            # Validate that both frames are valid
            if not aligned_depth_frame or not color_frame:
                continue

            depth_image = np.asanyarray(aligned_depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # Remove background - Set pixels further than clipping_distance to grey
            black_color = 0
            depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
            bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), black_color, color_image)

            # Render images:
            #   depth align to color on left
            #   depth on right
            #depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            #images = np.hstack((bg_removed, depth_colormap))
            cv2.namedWindow('depth_cut', cv2.WINDOW_NORMAL)
            cv2.imshow('depth_cut', bg_removed)
            camera_info = aligned_depth_frame.profile.as_video_stream_profile().intrinsics
            if cv2.waitKey(1) & 0xFF==27:
                capture=bg_removed
                cv2.destroyAllWindows()
                break

finally:
    while True:
        cv2.imshow('object',capture)
        cv2.setMouseCallback('object',distance)
        if cv2.waitKey(10) & 0xFF==27:
            cv2.destroyAllWindows()
            break
    # Stop streaming
    rtde_c.disconnect()
    pipeline.stop()

D400
There is a depth camera with color sensor
Depth Scale is:  0.0005000000237487257
599.9999715015306
00: Custom
01: Default
02: Hand
03: High Accuracy
04: High Density
1
(336, 199)
0.2175000160932541
Camera Points are: -0.03362567350268364 0.018777625635266304 -0.2175000160932541
[61.366259778862165, -706.5476424753098, 134.88266467223033, 1.0]
1
(511, 189)
0.0
No depth info, You must use an window
1
(507, 183)
0.0
No depth info, You must use an window
1
(504, 194)
0.0
No depth info, You must use an window
1
(529, 189)
0.0
No depth info, You must use an window
1
(527, 172)
0.0
No depth info, You must use an window
1
(561, 265)
0.18900001049041748
Camera Points are: 0.04092593491077423 -0.004272083751857281 -0.18900001049041748
[-16.825685725454818, -684.2645804461487, 132.20062908469077, 1.0]
1
(683, 213)
0.0
No depth info, You must use an window
1
(747, 244)
0.0
No depth info, You must use an window
1
(694, 270)
0.0
No depth info, You must use an window
1
(713, 124)
0.2915000021457

In [ ]:
a=[1.55, 5.66, 5.22, 1.0]
b=[1.2,5,2]
k=[a[0]/10.0,a[1]/10.0,b[0],b[1]]
print(k)